In [ ]:
import os
import json
import numpy as np
import pandas as pd
from datetime import datetime

exportname = "3d-art-blender-landmarks"        #Insert the Channel Name here
working_directory = os.getcwd()
#slackexport_folder_path = f"{working_directory}/{exportname}"
slackexport_folder_path = "/home/agds/Documents/RebeccaEverleneTrust/RebeccaEverlene_Slack_export"  #Insert path where the local copy of the GoogleDrive folder is.

# 1. Retrieving all information from Slack's channels and users:

The code presented in this section was developed by Rutvikk Kharod (https://github.com/Rutvikk-Khar/Data-Handling-of-Slack-JSON-Exports). There are minor modifications with respect to the original version, mainly to include some extra information in the dataframes and the further csv files.

If you are running this code for the first time, go ahead an execute the cells in sections 1.1, 1.2 and 1.3. to extract all the information contained in the exported file "RebeccaEverlene_Slack_export". On the other hand, if you have already ran this code, and have the csv files with all the Slack messages and users, then you can skip to Section 2.

## 1.1. Slack channels:
Reads the json file "channels.json", transfers the general information of each Slack channel to a dataframe and writes it to the "all_channels.csv" file.

In [ ]:
channels_path = f"{slackexport_folder_path}/channels.json"
with open(channels_path, encoding='utf-8') as f:
    channels_json = json.load(f)

channel_list = pd.DataFrame(columns=["ch_id", "name", "created", "creator", "is_archived",
                                     "is_general", "members", "topic", "purpose"])

for channel in range(len(channels_json)):
    channel_list.at[channel, "ch_id"] = channels_json[channel]['id']
    channel_list.at[channel, "name"] = channels_json[channel]['name']
    channel_list.at[channel, "created"] = channels_json[channel]['created']
    channel_list.at[channel, "creator"] = channels_json[channel]['creator']
    channel_list.at[channel, "is_archived"] = channels_json[channel]['is_archived']
    channel_list.at[channel, "is_general"] = channels_json[channel]['is_general']
    memberlist = ", ".join(channels_json[channel]['members'])
    channel_list.at[channel, "members"] = memberlist
    channel_list.at[channel, "topic"] = channels_json[channel]['topic']['value']
    channel_list.at[channel, "purpose"] = channels_json[channel]['purpose']['value']

    channel_folder_path = f"{slackexport_folder_path}/{channel_list.at[channel, 'name']}"
    channels_json[channel]['dayslist'] = os.listdir(channel_folder_path)

slack_export_channel_filename = "all_channels.csv"
channel_list.to_csv(slack_export_channel_filename, index=False)

In [ ]:
channel_list[:5]

## 1.2. Slack messages:
Collects all the messages in "RebeccaEverlene_Slack_export", stores them on the data frame "all_messages_df" and writes it into the file "all_channels_{Begin_Date}_{End_Date}.csv":

In [ ]:
def slack_json_to_dataframe(slack_json):
    messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text", "reply_count",
                                         "reply_users_count", "ts_latest_reply", "ts_thread", 
                                         "parent_user_id"])
    for message in range(len(slack_json)):
        if 'files' in slack_json[message] and slack_json[message]['files']:
            messages_df.at[message, "msg_id"] = slack_json[message]['files'][0]['id']
        elif 'client_msg_id' in slack_json[message]:
            messages_df.at[message, "msg_id"] = slack_json[message]['client_msg_id']
        if 'ts' in slack_json[message]:
            messages_df.at[message, "ts"] = slack_json[message]['ts']
        else:
            messages_df.at[message, "ts"] = None
        messages_df.at[message, "user"] = slack_json[message].get('user', None)
        if 'type' in slack_json[message]:
            messages_df.at[message, "type"] = slack_json[message]['type']
        else:
            messages_df.at[message, "type"] = None
        
        if 'text' in slack_json[message]:
            messages_df.at[message, "text"] = slack_json[message]['text']
        else:
            messages_df.at[message, "text"] = None
        if 'reply_count' in slack_json[message]:
            messages_df.at[message, "reply_count"] = slack_json[message]['reply_count']
            messages_df.at[message, "reply_users_count"] = slack_json[message]['reply_users_count']
            messages_df.at[message, "ts_latest_reply"] = slack_json[message]['latest_reply']
        if 'parent_user_id' in slack_json[message]:
            messages_df.at[message, "ts_thread"] = slack_json[message]['thread_ts']
            messages_df.at[message, "parent_user_id"] = slack_json[message]['parent_user_id']
    return messages_df

all_messages_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text",
                                                  "reply_count", "reply_users_count", 
                                                  "ts_latest_reply", "ts_thread", "parent_user_id", 
                                                  "channel", "json_name", "json_mod_date"])

for channel in range(len(channels_json)):
    all_channel_files_df = pd.DataFrame(columns=["msg_id", "ts", "user", "type", "text",
                                                 "reply_count", "reply_users_count", 
                                                 "ts_latest_reply", "ts_thread", "parent_user_id", "json_name", "json_mod_date"])

    for file_day in range(len(channels_json[channel]['dayslist'])):
        parentfolder_path = f"{slackexport_folder_path}/{channels_json[channel]['name']}"
        filejson_path = f"{parentfolder_path}/{channels_json[channel]['dayslist'][file_day]}"
        #print(channels_json[channel]['name'], channels_json[channel]['dayslist'][file_day] )
        with open(filejson_path, encoding='utf-8') as f:
            import_file_json = json.load(f)
        import_file_df = slack_json_to_dataframe(import_file_json)
        import_file_df['json_name'] = channels_json[channel]['dayslist'][file_day]#[:-5]
        import_file_df['json_mod_date'] = datetime.fromtimestamp( os.path.getmtime(filejson_path) )
        #print(channels_json[channel]['dayslist'][file_day], datetime.fromtimestamp( os.path.getmtime(filejson_path) ))
        all_channel_files_df = pd.concat([all_channel_files_df, import_file_df], ignore_index=True)

    all_channel_files_df['channel'] = channels_json[channel]['name']
    #all_channel_files_df['file_name'] = 
    all_messages_df = pd.concat([all_messages_df, all_channel_files_df], ignore_index=True)

### To write a .csv file with all the channels:
all_messages_mindate = pd.to_datetime(all_messages_df['ts'], unit='s').min().date()
all_messages_maxdate = pd.to_datetime(all_messages_df['ts'], unit='s').max().date()
all_messages_filename = f"{"all_channels"}_{all_messages_mindate}_to_{all_messages_maxdate}.csv"  
all_messages_df.to_csv(all_messages_filename, index=False)

In [ ]:
all_messages_df[:3]

## 1.3. Slack users:
Reads the json file "users.json", transfers the general information of each Slack user to a dataframe and writes it to the "all_users.csv" file.

In [ ]:
users_path = f"{slackexport_folder_path}/users.json"
with open(users_path, encoding='utf-8') as f:
    users_json = json.load(f)

user_list_df = pd.DataFrame(columns=["user_id", "team_id", "name", "deleted", "real_name",
                                      "tz", "tz_label", "tz_offset", "title", "display_name", 
                                      "is_bot"])

for user in range(len(users_json)):
    user_list_df.at[user, "user_id"] = users_json[user]['id']
    user_list_df.at[user, "team_id"] = users_json[user]['team_id']
    user_list_df.at[user, "name"] = users_json[user]['name']
    user_list_df.at[user, "deleted"] = users_json[user]['deleted']
    user_list_df.at[user, "real_name"] = users_json[user].get('real_name', None)
    user_list_df.at[user, "title"] = users_json[user]['profile']['title']
    user_list_df.at[user, "display_name"] = users_json[user]['profile']['display_name']
    user_list_df.at[user, "is_bot"] = users_json[user]['is_bot']
    user_list_df.at[user, "tz"] = users_json[user].get('tz', None)
    user_list_df.at[user, "tz_label"] = users_json[user].get('tz_label', None)
    user_list_df.at[user, "tz_offset"] = users_json[user].get('tz_offset', None)

slack_export_user_filename = "all_users.csv"
user_list_df.to_csv(slack_export_user_filename, index=False)

In [ ]:
user_list_df[:3]

# 2. Compiling information for a given Slack channel:
The three csv files created above contain all the information for all the Slack users and channels. They can be generated once and used as many times as needed for more especialize queries. 

If you already have the general .csv files saved in your working directory, you can import them into dataframes by executing the following cell:

In [ ]:
all_messages_df = pd.read_csv(f"{working_directory}/all_channels_2021-04-30_to_2024-10-03.csv")    
user_list_df    = pd.read_csv(f"{working_directory}/all_users.csv")

Otherwise, continue with the cells below:

In [ ]:
### Write a .csv file with the same format as all_channels_2021-04-30_to_2024-10-03.csv but only containing the messages of the 
### desire channel:
exportChannel_messages_df = all_messages_df[all_messages_df['channel']==exportname]
exportChannel_mindate = pd.to_datetime(exportChannel_messages_df['ts'], unit='s').min().date()
exportChannel_maxdate = pd.to_datetime(exportChannel_messages_df['ts'], unit='s').max().date()
exportChannel_messages_filename = f"{exportname}_{exportChannel_mindate}_to_{exportChannel_maxdate}.csv"
exportChannel_messages_df.to_csv(exportChannel_messages_filename, index=False)

#### Create a curated dataframe with the users information from the desired channel:
channel_users = []
for user in exportChannel_messages_df['user'].unique():
    ## Get information from the users dataframe:
    if user == "USLACKBOT":
        user_id = "Slackbot"
        team_id = ''
        name = "Slackbot"
        display_name = "Slackbot"
    else:
        user_df = user_list_df[user_list_df['user_id']==user][['user_id','team_id','name', 'display_name']]
        user_id = user_df['user_id'].values
        team_id = user_df['team_id'].values
        name = user_df['name'].values
        display_name = user_df['display_name'].values

    channel_users.append( np.array([user_id, team_id, name, display_name]).flatten() ) 
channel_users = pd.DataFrame(channel_users, columns=['user_id', 'team_id', 'name', 'display_name'])

### Use the curated channel_users dataframe to fill-in the user's information for each message:
for index in exportChannel_messages_df.index.values:
    #exportChannel_messages_df.at[index, 'user_id'] = channel_users[channel_users['user_id']==exportChannel_messages_df.loc[index]['user']]['user_id'].values
    #exportChannel_messages_df.at[index, 'team_id'] = channel_users[channel_users['user_id']==exportChannel_messages_df.loc[index]['user']]['team_id'].values
    exportChannel_messages_df.at[index, 'name'] = channel_users[channel_users['user_id']==exportChannel_messages_df.loc[index]['user']]['name'].values
    exportChannel_messages_df.at[index, 'display_name'] = channel_users[channel_users['user_id']==exportChannel_messages_df.loc[index]['user']]['display_name'].values

exportChannel_messages_df[:3]

In [ ]:
### Drop unnecessary columns:
exportChannel_messages_df.drop(['reply_count', 'reply_users_count', 'ts_latest_reply', 'ts_thread', 'parent_user_id'], axis=1, inplace=True)
exportChannel_messages_df[:3]

In [ ]:
### Reorder the columns:
exportChannel_messages_df = exportChannel_messages_df[['channel', 'json_name', 'json_mod_date', 'user', 'name', 'display_name', 'ts', 'msg_id', 'type', 'text']]
exportChannel_messages_df.index = ['']*len(exportChannel_messages_df)
exportChannel_messages_df[:3]

In [ ]:
### Change format of the time in seconds to a date:
exportChannel_messages_df['ts'] = pd.to_datetime(exportChannel_messages_df['ts'], unit='s')
exportChannel_messages_df.rename(columns={"ts": "msg_date"}, inplace=True)
exportChannel_messages_df.sort_values(by='msg_date', inplace=True)
exportChannel_messages_df[:3]

In [ ]:
### Write .csv file:
cleanExportChannel_filename = f"{exportname}_{exportChannel_mindate}_to_{exportChannel_maxdate}_compiled.csv"
exportChannel_messages_df.to_csv(cleanExportChannel_filename, index=False)